# Nanoclusters of Gold

## 0. Introduction

This notebook demonstrates creation of Gold nanoclusters with cuboctahedral and icosahedral shapes. The notebook is based on the following publication:


> **A. H. Larsen, J. Kleis, K. S. Thygesen, J. K. Nørskov, and K. W. Jacobsen**,
> "Electronic shell structure and chemisorption on gold nanoparticles",
> *Phys. Rev. B 84, 245429 (2011)*,
> [DOI: 10.1103/PhysRevB.84.245429](https://doi.org/10.1103/PhysRevB.84.245429)
> 

In this notebook we will reproduce materials from FIG. 2:

![Fig 2.](https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/0d_materials/nanocluster_gold/0-manuscript-image.webp
)


## 1. Prepare the Environment


### 1.1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("https://exabyte-io.github.io/api-examples/mat3ra_api_examples-0.1.dev1+g2d074bd-py3-none-any.whl", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples")

### 1.2. Set up cluster parameters
Parameters for the cuboctahedron and icosahedron shapes can be found at [ASE Clusters](https://wiki.fysik.dtu.dk/ase/ase/cluster/cluster.html#ase.cluster.Cluster).

In [ ]:
from types import SimpleNamespace
from mat3ra.made.tools.build.nanoparticle.enums import NanoparticleShapesEnum as Shapes

ELEMENT = "Au"  # Chemical symbol of the element to create nanoparticles from


def length(cutoff):
    # The parameters of to achieve cuboctahedron shape from the octahedron
    return 2 * cutoff + 1


NANOPARTICLE_CONFIGS = [
    SimpleNamespace(
        shape=Shapes.OCTAHEDRON,
        parameters=SimpleNamespace(
            length=length(2),
            cutoff=2,
        )
    ),
    SimpleNamespace(
        shape=Shapes.OCTAHEDRON,
        parameters=SimpleNamespace(
            length=length(3),
            cutoff=3,
        )
    ),
    SimpleNamespace(
        shape=Shapes.OCTAHEDRON,
        parameters=SimpleNamespace(
            length=length(4),
            cutoff=4,
        )
    ),
    SimpleNamespace(
        shape=Shapes.ICOSAHEDRON,
        parameters=SimpleNamespace(
            noshells=3,
        )
    ),
    SimpleNamespace(
        shape=Shapes.ICOSAHEDRON,
        parameters=SimpleNamespace(
            noshells=4,
        )
    ),
    SimpleNamespace(
        shape=Shapes.ICOSAHEDRON,
        parameters=SimpleNamespace(
            noshells=5,
        )
    )
]


### 1.3. Get input material

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

material = Material.create(Materials.get_by_name_first_match("Gold"))

## 2. Create the Target Material(s)

In [ ]:
from mat3ra.made.tools.helpers import create_nanoparticle_by_shape

nanoparticles = [create_nanoparticle_by_shape(
    crystal=material,
    shape=config.shape,
    parameters=config.parameters
) for config in NANOPARTICLE_CONFIGS]

## 3. Visualize the Result(s)

In [ ]:
from utils.visualize import visualize_materials as visualize

cuboctahedra_particles = nanoparticles[0:3]
icosahedra_particles = nanoparticles[3:6]
visualize(cuboctahedra_particles, rotation="45y,45x")
visualize(icosahedra_particles, rotation="60x")

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials, download_content_to_file

set_materials(nanoparticles)
for idx, nanoparticle in enumerate(nanoparticles):
    download_content_to_file(nanoparticle.to_json(), f"gold_nanocluster_{idx}.json")